In [1]:
# Librerias
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

import statsmodels.api as sm
from statsmodels.tools import eval_measures


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV


In [2]:
# leemos el dataset limpio 
filename = "../dataset/capital.csv"
data_original = pd.read_csv(filename, sep=',')
display(data_original.sample(n=5))
print(f"(Cantidad de Filas, Cantidad de columnas) -> {data_original.shape}")

,Unnamed: 0,numero,property_type,place_name,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,piscina,Parrilla,Cochera,nuevo,description,title
2459,2459,2460,store,Recoleta,6352020,360000,30,30,12000,12000,1,1,30000,0,0,0,0,Local comercial a la calle en excelente zona ...,Local - Recoleta
12063,12063,12064,apartment,Caballito,1764450,100000,59,29,1695,3448,1,1,0,0,1,0,0,- Ascensor automático con cabina de acero inox...,Departamento 1 ambiente con balcón y terraza p...
5011,5011,5012,apartment,San Telmo,16991345,962983,44,35,21886,27514,6,2,0,0,1,1,0,Formando Parte del Barrio de San Telmo: Caract...,Departamento - 2 Ambientes - San Telmo
16717,16717,16718,apartment,Recoleta,5205128,295000,77,77,3831,3831,3,4,0,0,0,0,0,"Plena Recoleta, living comedor, comedor diario...",Departamento en Recoleta
7974,7974,7975,apartment,Caballito,22895680,129761,43,39,3018,3327,16,1,0,1,1,1,1,VENTA DEPARTAMENTO 1 AMBIENTE CABALLITOUnidad ...,DEPARTAMENTO EN VENTA


(Cantidad de Filas, Cantidad de columnas) -> (16938, 19)


Utilizaremos unicamente datos de apartamentos en palermo

In [3]:
data=data_original[(data_original.property_type=='apartment') & (data_original.nuevo==0) &(data_original.price_aprox_usd<800000)
 & (data_original.surface_total_in_m2<500) & (data_original.surface_covered_in_m2<500) & (data_original.rooms>=1) 
 &(data_original.floor>=1) & (data_original.place_name=='Capital Federal')]
display(data.sample(2))
data.shape

,Unnamed: 0,numero,property_type,place_name,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,piscina,Parrilla,Cochera,nuevo,description,title
4380,4380,4381,apartment,Capital Federal,2117340,120000,62,61,1935,1967,3,3,12000,0,0,0,0,DUEÑO VENDE HERMOSO 3 AMB + ENTREPISO. EXCELEN...,Departamento venta
13827,13827,13828,apartment,Capital Federal,4499348,255000,116,95,2198,2684,2,8,0,0,0,0,0,"Se trata de un departamento al frente, de 8 am...",Departamento - Recoleta


(120, 19)

In [4]:
#Eliminamos columnas que no vamos a usar
data.drop([data.columns[0],"numero","description", "price_usd_per_m2", "price_aprox_local_currency","price_per_m2" ,"title", "property_type"], axis=1,inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_8268\67199441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop([data.columns[0],"numero","description", "price_usd_per_m2", "price_aprox_local_currency","price_per_m2" ,"title", "property_type"], axis=1,inplace=True)


In [5]:
data.shape

(120, 11)

# Dataset

El dataset ya se encuentra limpio, y cuenta con 120 Propiedades de Palermo.

**Campos**
<pre>
place_name:                Barrio
price_aprox_usd:           Precio en dolares. Variable Target
surface_total_in_m2:       Metros cuadrados totales de la propiedad
surface_covered_in_m2:     Metros cuadrados cubiertos de la propiedad
floor:                     Piso en que se encuentra la propiedad.
rooms:                     Habitaciones de la propiedad
expenses:                  Valor de las Expensas en pesos
piscina:                   Si tiene pisina (0: No tiene, 1: Tiene)
Parrilla:                  Si tiene parrilla (0: No tiene, 1: Tiene)
Cochera:                   Si tiene cochera (0: No tiene, 1: Tiene)
nuevo:                     Si es nuevo (0: no es nuevo, 1: es nuevo)
</pre>

`place_name` es una Variable categorica,hay que crear variables dummies.
`surface_total_in_m2`, `surface_covered_in_m2`, `floor`, `rooms` y `expenses` son variables continuas, y hay que estandarizarlas para llevarlas al intevalo 0-1.


In [6]:
categoricals = ['place_name']

enc = OneHotEncoder(drop='first')
X = data[categoricals]
enc.fit(X)
enc.categories_

[array(['Capital Federal'], dtype=object)]

In [7]:
dummies = enc.transform(X).toarray()
dummies

array([], shape=(120, 0), dtype=float64)

In [8]:
dummies.shape

(120, 0)

In [9]:
# Convertimos en dataframe por comodidad.
dummies_df = pd.DataFrame(dummies)
dummies_df

""
0
1
2
3
4
...
115
116
117
118


In [10]:
col_names = [categoricals[i] + '_' + enc.categories_[i] for i in range(len(categoricals)) ]
col_names_drop_first = [sublist[i] for sublist in col_names for i in range(len(sublist)) if i != 0]
dummies_df.columns = col_names_drop_first
dummies_df

""
0
1
2
3
4
...
115
116
117
118


Paso 2:
Estandarizamos los campos numéricos `surface_total_in_m2`, `surface_covered_in_m2`, `floor`, `rooms` y `expenses`.

In [11]:
numericals = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms', 'expenses']

X = data[numericals]

scaler = MinMaxScaler()
scaler.fit(X)

std_numerical_data = scaler.transform(X)
std_df = pd.DataFrame(std_numerical_data)
std_df.columns = [i + '_std' for i in numericals]
std_df

,surface_total_in_m2_std,surface_covered_in_m2_std,floor_std,rooms_std,expenses_std
0,0.169972,0.195349,0.875,0.125,0.000000
1,0.283286,0.325581,0.250,0.250,0.125000
2,0.056657,0.093023,0.125,0.000,0.000000
3,0.084986,0.139535,0.000,0.125,0.333333
4,0.084986,0.120930,0.250,0.250,0.141667
...,...,...,...,...,...
115,0.184136,0.246512,0.125,0.125,0.000000
116,0.118980,0.144186,0.125,0.000,0.000000
117,0.161473,0.227907,0.250,0.125,0.000000
118,0.184136,0.255814,0.875,0.000,0.000000


Ahora nuestro dataset de features serán las variables dummies y las variables numéricas estandarizadas

In [12]:
X = pd.concat([dummies_df, std_df], axis = 1)
y = data.price_aprox_usd  #variable target

Paso 3:
Separamos el conjunto train y test.
* Spliteamos el dataframe el dataframe construido, con las variables dummies y estandarizadas.
* El tamaño del conjunto de datos lo definimos en 30% de los datos.
* fijamos la semilla de los nímeros aleatoaleatoriosrios.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 117)

Paso 4:
Ajustemos una regresión lineal múltiple con los datos del conjunto de entrenamiento usando statsmodels y evaluemos la significancia de cada uno de los coeficientes

In [ ]:
###### NO EJECUTA ESTA LINEA  ######
####################################

X_train_sm = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_sm).fit()
print(model.summary())

In [ ]:
no_reg_model_params = model.params

calculamos el error cuadratico en train y test.

In [ ]:
sm_prediction_train = model.predict(X_train_sm)
print(eval_measures.rmse(y_train, sm_prediction_train))

X_test_sm = sm.add_constant(X_test)
sm_prediction_test = model.predict(X_test_sm)
print(eval_measures.rmse(y_test, sm_prediction_test))

Si los resultados no son parecidos, no fueron bien entrenados.

Si el `R-squared` y el `R-squared ajustado` dan muy bajos, creemos que usando regularizaciòn podrìamos arreglar este problema.
<pre>
R-squared:	0.175    ##AREGLAR
Adj. R-squared:	0.170 ##ARREGLAR
</pre>

Paso 5:

Ajustamos el modelo aplicando regularización de Lasso y validación cruzada para estimar el mejor valor de $\alpha$ para este problema

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Definimos el rango de de búsqueda del hiperparametro explicitamente
# normalize = False, porque ya normalizamos.
# cv=5 -> Dividimos el entrenamiento en 5.
lm_lasso = linear_model.LassoCV(alphas=[0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01,\
                                        0.05, 0.1, 1, 5, 10, 15, 30, 40, 50, 100],\
                                        normalize=False, cv=5) 

model_cv = lm_lasso.fit(X_train, y_train)

model_cv.score(X_train, y_train)

In [ ]:
model_cv.coef_

In [ ]:
model_cv.intercept_

In [ ]:
model_cv.alpha_

Se planteron todos los valores de alfa posibles,
Como dividimos el entrenamiento en 5, se calcula el error cuadratico promedio de los 5 entrenamientos, y obtenemos el Alfa.
Ahora chequeamos que el resultado no sea uno de los valores extremos: En este caso, `0.1750431565177899` ##AREGLAR no es extremo, por lo que lo tomamos como válido y vamos a confiar en ese alfa

In [ ]:
model_cv.score(X_train, y_train)

Paso 6:

Ajustemos los datos de entrenamiento con una regresión con regularización de Lasso para el valor de $\alpha$ calculado en el punto anterior usando statsmodels.

Usemos scatterplots para mostrar 

* los valores de los coeficientes de la regresión lineal múltiple obtenidos en el Paso 3, y los valores de los coeficientes de la regresión lineal con regularización de Lasso para el modelo entrenado.

* los valores de los residuos en entrenamiento resultado del Paso 3, y los residuos en entrenamiento para el modelo con regularización.

In [ ]:
best_alpha = model_cv.alpha_

#L1_wt : 0, the fit is ridge regression. 1, the fit is the lasso 

no_reg_model = sm.OLS(y_train, X_train_sm)

reg_model = no_reg_model.fit_regularized(alpha = best_alpha, L1_wt = 1)

In [ ]:
reg_model.params

Podemos ver que lasso hizo cero, elimino la variable expensas, ya que cree que no es buena para el modelo.  ##ver si es real

In [ ]:
sns.scatterplot(x=reg_model.params, y=no_reg_model_params);

In [ ]:
reg_residuals = y_train - reg_model.fittedvalues
linear_residuals = y_train - model.fittedvalues
sns.scatterplot(x = reg_residuals, y = linear_residuals)

Paso 7:

Usamos statsmodels y scikit-learn para calcular la performance en test del modelo construído y comparemos los resultados de las dos bibliotecas usando como métricas el error absoluto medio (MAE) y la raiz del error cuadrático medio (RMSE) 

In [ ]:
sm_prediction = reg_model.predict(X_test_sm)
sm_prediction

In [ ]:
skl_lasso = linear_model.Lasso(alpha = best_alpha, fit_intercept=True, normalize=False)
skl_lasso = skl_lasso.fit(X= X_train, y = y_train)
skl_prediction = skl_lasso.predict(X_test)

skl_residuals = y_test - skl_prediction
sm_residuals = y_test - sm_prediction
sns.scatterplot(x = skl_residuals, y = sm_residuals)

In [ ]:
lasso_coef = np.insert(skl_lasso.coef_, 0, skl_lasso.intercept_)
sns.scatterplot(x = lasso_coef, y = reg_model.params);

In [ ]:
lasso_coef

In [ ]:
reg_model.params

Métricas en `statsmodels`

In [ ]:
eval_measures.rmse(y_test, sm_prediction)

In [ ]:
eval_measures.meanabs(y_test, sm_prediction)

In [ ]:
# de scikit-learn
metrics.r2_score(y_test, sm_prediction)

Métricas en `scikit-learn`


In [ ]:
np.sqrt(metrics.mean_squared_error(y_test, skl_prediction))

In [ ]:
metrics.mean_absolute_error(y_test, skl_prediction)

In [ ]:
metrics.r2_score(y_test, skl_prediction)

In [ ]:
o = data[:20][['place_name', 'price_aprox_usd']]
p = skl_lasso.predict(X_test[:20])
for idx in range(len(p)):
    print(f'Localidad: {o.place_name[idx]:<25}Precio dataset: {o.price_aprox_usd[idx]:<8}Precio predecido:{p[idx]:<25}Diferencia: {p[idx]-o.price_aprox_usd[idx]:.0f}')

In [ ]:
o = data[['place_name', 'price_aprox_usd']]
p = skl_lasso.predict(X_test)
for idx in range(len(p)):
    if abs(p[idx] - o.price_aprox_usd[idx])<5000:
        print(f'Localidad: {o.place_name[idx]:<25}Precio dataset: {o.price_aprox_usd[idx]:<8}Precio predecido:{p[idx]:<25}Diferencia: {p[idx]-o.price_aprox_usd[idx]:.0f}')

RIDGE

In [ ]:
#Paso 2: Establecer features y target
X = pd.concat([dummies_df, std_df], axis = 1)
y = data.price_aprox_usd


In [ ]:
numericals = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms', 'expenses']

X = data[numericals]

scaler = MinMaxScaler()
scaler.fit(X)

std_numerical_data = scaler.transform(X)
std_df = pd.DataFrame(std_numerical_data)
std_df.columns = [i + '_std' for i in numericals]
std_df

In [ ]:
#Paso 3:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 117)

In [ ]:
#Paso 4: Calculamos intervalos de confianza
X_train_sm = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_sm).fit()
model.summary()

In [ ]:
no_reg_model_params = model.params

In [ ]:
#calculo error cuadratico en train y test:
sm_prediction_train = model.predict(X_train_sm)
print(eval_measures.rmse(y_train, sm_prediction_train))

In [ ]:
model_ridge_cv = RidgeCV(alphas= [0.3, 0.5, 1.0, 1.1, 1.15, 1.17, 1.18, 1.19, 1.2, 1.21, 1.22, 1.3, 1.4, 1.5, 10.0], 
                                   fit_intercept=True, cv=10)
model_ridge_cv.fit(X_train_sm, y_train)

print(model_ridge_cv.alpha_)

print(model_ridge_cv.coef_)

print(model_ridge_cv.intercept_)


In [ ]:
#Comparo metrica de train y test:
print ('Modelo Ridge:\n')
print (f'Score (R^2) en train del modelo es {model_ridge_cv.score(X_train_sm, y_train)} y en test es {model_ridge_cv.score(X_test_sm, y_test)}')
print ('\n')

In [ ]:
X_test_sm = sm.add_constant(X_test)
sm_prediction_test = model.predict(X_test_sm)
print(eval_measures.rmse(y_test, sm_prediction_test))

In [ ]:
#Paso 5:
import warnings
warnings.filterwarnings('ignore')

Por falta de tiempo no logramos terminar este modelo 